In [1]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
!pip install Unidecode
try:
    import re2 as re
except ImportError:
    import re
import numpy as np
from unidecode import unidecode
import os
from collections import Counter
from keras.models import load_model
from numpy import array
from numpy import argmax
from nltk import ngrams
from nltk.translate.bleu_score import sentence_bleu
import string
import math

path_checkpoint = '/content/drive/My Drive/VNTones/model'

     |████████████████████████████████| 245kB 2.8MB/s 


Using TensorFlow backend.


In [3]:
accented_chars_vietnamese = [
    'á', 'à', 'ả', 'ã', 'ạ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ',
    'ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ',
    'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ',
    'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự',
    'í', 'ì', 'ỉ', 'ĩ', 'ị',
    'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ',
    'đ',
]
accented_chars_vietnamese.extend([c.upper() for c in accented_chars_vietnamese])
alphabet = list(('\x00 _' + string.ascii_letters + string.digits + ''.join(accented_chars_vietnamese)))



s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'


In [4]:
if (os.path.exists(path_checkpoint)):
    listEpochs = [x for x in os.listdir(path_checkpoint) if x[:6] == 'model_' and x[-3:] == '.h5']
    lastEpoch = max([int(x[-6:-3]) for x in listEpochs])
    lastEpochFile = path_checkpoint + '/' + 'model_%03d.h5' % (lastEpoch)
lastEpochFile = '/content/drive/My Drive/VNTones/model/model_046.h5'

model = load_model(lastEpochFile)

In [7]:

NGRAM = 5
MAXLEN = 30

def remove_accents(input_str):
	s = ''
	for c in input_str:
		if c in s1:
			s += s0[s1.index(c)]
		else:
			s += c
	return s

def accent_sentence(sentence):
  list_phrases = extract_phrases(sentence)
  output = ()
  for phrases in list_phrases:
    if len(phrases.split()) < 2 or not re.match("\w[\w ]+", phrases):
      output += phrases
    else:
      outputs = add_accent(phrases)
      output = outputs
      # output += add_accent(phrases)
      if phrases[-1] == " ":
        output += " "

  return output

def text_cleaner(text):
    text = re.sub(r"'s\b","",text)
    # remove punctuations
    # INTAB = "ạảãàáâậầấẩẫăắằặẳẵóòọõỏôộổỗồốơờớợởỡéèẻẹẽêếềệểễúùụủũưựữửừứíìịỉĩýỳỷỵỹđẠẢÃÀÁÂẬẦẤẨẪĂẮẰẶẲẴÓÒỌÕỎÔỘỔỖỒỐƠỜỚỢỞỠÉÈẺẸẼÊẾỀỆỂỄÚÙỤỦŨƯỰỮỬỪỨÍÌỊỈĨÝỲỶỴỸĐ"
    text = re.sub("[^a-zA-Z0-9ạảãàáâậầấẩẫăắằặẳẵóòọõỏôộổỗồốơờớợởỡéèẻẹẽêếềệểễúùụủũưựữửừứíìịỉĩýỳỷỵỹđẠẢÃÀÁÂẬẦẤẨẪĂẮẰẶẲẴÓÒỌÕỎÔỘỔỖỒỐƠỜỚỢỞỠÉÈẺẸẼÊẾỀỆỂỄÚÙỤỦŨƯỰỮỬỪỨÍÌỊỈĨÝỲỶỴỸĐ ]", "", text)
    return text

def encode(text, maxlen=MAXLEN):
        text = "\x00" + text
        x = np.zeros((maxlen, len(alphabet)))
        for i, c in enumerate(text[:maxlen]):
            x[i, alphabet.index(c)] = 1
        if i < maxlen - 1:
          for j in range(i+1, maxlen):
            x[j, 0] = 1
        return x

def gen_list(ngrams_list):
    len_ngrams_list = len(ngrams_list)
    number_candidate = len(ngrams_list[0])
    index_arr = [[] for i in range(len_ngrams_list)]
    prob_arr = [[] for i in range(len_ngrams_list)]
    
    for i in range(len_ngrams_list):
        for j in range(number_candidate):
            index_arr[i].append(ngrams_list[i][j][0])
            prob_arr[i].append(ngrams_list[i][j][1])
    # print(index_arr)
    # print(prob_arr)
    return index_arr, prob_arr

def extract_phrases(text):
    pattern = r'\w[\w ]*|\s\W+|\W+'
    return re.findall(pattern, text)
  
def gen_ngrams(words, n=5):
    return ngrams(words.split(), n)

def decode(x, calc_argmax=True):
  if calc_argmax:
      x = x.argmax(axis=-1)
  return ''.join(alphabet[i] for i in x)
  
def beam_search_decoder(data, k):
	sequences = [[list(), 0.0]]
	# walk over each step in sequence
	for row in data:
		all_candidates = list()
		# expand each current candidate
		for i in range(len(sequences)):
			seq, score = sequences[i]
			for j in range(len(row)):
# 				print(j)
# 				print(row[j] - 1e-30 < 0)
				candidate = [seq + [j], score - math.log(row[j])]
				all_candidates.append(candidate)
		# order all candidates by score
		ordered = sorted(all_candidates, key=lambda tup:tup[1])
		# select k best
		sequences = ordered[:k]
	return sequences

# beam search
def beam_search_decoder2(data, k):
	sequences = [[list(), 0.0]]
	# walk over each step in sequence
	for row in data:
		all_candidates = list()
		# expand each current candidate
		for i in range(len(sequences)):
			seq, score = sequences[i]
			for j in range(len(row)):
				candidate = [seq + [j], score - math.log(row[j])]
				all_candidates.append(candidate)
		# order all candidates by score
		ordered = sorted(all_candidates, key=lambda tup:tup[1])
		# select k best
		sequences = ordered[:k]
	return sequences
  
def decode_bs(data):
  res = beam_search_decoder(data, 3)
  for i, seq in enumerate(res):
      res[i][0] = ''.join(alphabet[k] for k in seq[0]).strip('\x00')
  return res



def guess(ngram):
    text = ' '.join(ngram)
    # print(text)
    preds = model.predict(np.array([encode(text)]), verbose=0)
    # print(preds[0])
    return decode_bs(preds[0])
    # return decode(preds[0], calc_argmax=True)[1:len(text)+1], preds[0]


def candidates(guessed_ngrams):
  candidates = [Counter() for _ in range(len(guessed_ngrams) + NGRAM - 1)]
  for nid, ngram in enumerate(guessed_ngrams):
        for wid, word in enumerate(re.split(' +', ngram)):
            candidates[nid + wid].update([word])
  output = ' '.join(c.most_common(1)[0][0] for c in candidates)
  return output

def Mapping(a,b):
  for i in range(len(b)):
    for j in range(len(b[0])):
      b[i][j] = b[i][j]

  for i in range(len(b)):
    for j in range(len(a)):
      b[i][j] = a[j][b[i][j]]
  return b

def add_accent(text):
    ngrams = list(gen_ngrams(text, n=NGRAM))
    guessed_ngrams = []
    candidates_index =[]
    for ngram in ngrams:
      ngram_set = guess(ngram)
      print(ngram_set)
      guessed_ngrams.append(ngram_set)
    index_arr, prob_arr = gen_list(guessed_ngrams)

    for i in range(len(prob_arr)):
      for j in range(len(prob_arr[0])):
        prob_arr[i][j] = (1/prob_arr[i][j])

    # print(prob_arr)

    index_cand = beam_search_decoder2(prob_arr, 3)

    for i in range(len(index_cand)):
      candidates_index.append(index_cand[i][0])

    # print(candidates_index)
 
    candidates_index = Mapping(index_arr,candidates_index)
    a = [] 
    # print(candidates_index)
    for candidates_sentence in candidates_index:
      a.append(candidates(candidates_sentence))
    return (a)

text_1 = 'Điều này cũng hoàn toàn phù hợp với xu hướng phát triển của thị trường giai đoạn 3 năm qua'
text_1 = text_cleaner(text_1)

text_kodau = remove_accents(text_1)
print(text_kodau)
ref = [text_1.split()]
# print("result from "+lastEpochFile)
for i,result in enumerate(accent_sentence(text_kodau)[:3]):
  can = result.split()
  score = sentence_bleu(ref, can, weights=(1,0,0,0))
  print('kết quả dự đoán',i+1,': ',result,'===== Điểm BLEU:',score)

 
    


Dieu nay cung hoan toan phu hop voi xu huong phat trien cua thi truong giai doan 3 nam qua
[['Điều này cũng hoàn toàn', 0.016023754686108244], ['Điều này cùng hoàn toàn', 4.407390822956666], ['Điều nay cũng hoàn toàn', 6.613001127366269]]
[['này cũng hoàn toàn phụ', 0.5692254446064041], ['này cũng hoàn toàn phù', 1.248585873392355], ['này cũng hoàn toàn phủ', 2.509784856093999]]
[['cũng hoàn toàn phù hợp', 0.20133753756643924], ['cùng hoàn toàn phù hợp', 2.2028823981543098], ['cứng hoàn toàn phù hợp', 3.220324368636257]]
[['hoàn toàn phù hợp với', 0.0017168911433487928], ['hoàn toàn phụ hợp với', 7.72777006201997], ['hoàn toàn phủ hợp với', 8.01064329987076]]
[['toán phù hợp với xu', 1.099002481356898], ['toàn phù hợp với xu', 1.6795115931838915], ['toán phù hợp với xử', 1.8367820185748314]]
[['phù hợp với xu hướng', 0.0032918195285906527], ['phủ hợp với xu hướng', 6.342000680611386], ['phụ hợp với xu hướng', 7.866159130959694]]
[['hợp với xu hướng phát', 0.00797897041945364], ['hợp vớ

In [ ]:
!ls -la